<a href="https://colab.research.google.com/github/DevNicque/MyCods/blob/master/Vers%C3%A3o2_base_covid_vacina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Configuração**

In [1]:
!pip install unidecode
!pip install elasticsearch

     |████████████████████████████████| 245kB 9.2MB/s 
     |████████████████████████████████| 327kB 8.3MB/s 


In [2]:
import pandas as pd
from unidecode import unidecode
from datetime import date,datetime, timedelta
import pytz
from elasticsearch import helpers,Elasticsearch
import json

In [3]:
elasticsearch_username = 'elastic'
elasticsearch_password = 'gxjzXnMQ9ZGHE9Huc7'
elasticsearch_host = '159.89.82.132'
es = Elasticsearch(server_hostname=[elasticsearch_host],
    http_auth=(elasticsearch_username,
    elasticsearch_password),scheme="https",
    hosts=[elasticsearch_host],port=9300,
    verify_certs=False)

/usr/local/lib/python3.6/dist-packages/elasticsearch/connection/http_urllib3.py:209: UserWarning: Connecting to https://159.89.82.132:9300 using SSL with verify_certs=False is insecure.
  % self.host


In [4]:
def time_stamp():

        # a timestamp I'd like to convert
        my_timestamp = datetime.now()

        # create both timezone objects
        old_timezone = pytz.timezone("Universal")
        new_timezone = pytz.timezone("America/Manaus")#(AMT – Amazon Time em UTC-04:00)

        # two-step process
        #localized_timestamp = old_timezone.localize(my_timestamp)
        #new_timezone_timestamp = localized_timestamp.astimezone(new_timezone)

        # or alternatively, as an one-liner
        new_timezone_timestamp = old_timezone.localize(my_timestamp).astimezone(new_timezone)

        obj_str = new_timezone_timestamp.strftime("%Y-%m-%d %H:%M:%S")

        return obj_str

In [5]:
time_stamp()

'2021-02-22 19:26:52'

#**Importação de Dados**

##Base FVS Casos

dia anterior

In [68]:
base_casos_anterior = pd.ExcelFile('covid_fvs_casos_por_municipios_anterior.xlsx')
casos_anterior = pd.read_excel(base_casos_anterior, 'covid_por_municipios')

dia atual

In [69]:
base_casos_atual = pd.ExcelFile('covid_fvs_casos_por_municipios_atual.xlsx')
casos_atual = pd.read_excel(base_casos_atual, 'covid_casos_por_municipios')

##Base FVS Leitos

In [ ]:
base_leitos = pd.ExcelFile('covid_leitos.xlsx')
leitos = pd.read_excel(base_leitos, 'leitos')
taxa_ocupacao = pd.read_excel(base_leitos, 'taxa_ocupacao_geral')
capacidade_instalada = pd.read_excel(base_leitos, 'capacidade_instalada')


##Base Prefeitura Vacinados

In [ ]:
base_vacinados = pd.ExcelFile('vacinas_sepultamentos.xlsx')
vacinados_por_grupo = pd.read_excel(base_vacinados, 'grupo_vacinados')
vacinados_por_dia = pd.read_excel(base_vacinados, 'vacinados_por_dia')
vacinados_totais = pd.read_excel(base_vacinados, 'total_vacinados')
vacinados_por_local = pd.read_excel(base_vacinados, 'vacinados_local')
sepultamentos = pd.read_excel(base_vacinados, 'sepultamentos')

##Base Teixeira Predição

In [ ]:
predicao_casos = pd.read_csv('media_movel_casos_prevista_AI.csv')

In [ ]:
predicao_obitos = pd.read_csv('media_movel_deaths_prevista_AI.csv')

#**Tratamento**

##**Casos por Municipios**

###**atual**

In [70]:
casos_atual.head()

,Municípios,Notificados,Confirmados (3),% de participação\npor Município,Nº de óbitos,Incidência (casos/100.000),Mortalidade (óbitos/100.000),Letalidade,data_coleta
0,AMAZONAS,746528,306003,1.0000,10462,7383.18,252.43,0.0342,2021-02-21
1,MANAUS,384525,141674,0.4630,7407,6490.58,339.34,0.0523,2021-02-21
2,INTERIOR,362003,164329,0.5370,3055,8376.29,155.72,0.0186,2021-02-21
3,Coari,22800,8952,0.0293,169,10519.76,198.60,0.0189,2021-02-21
4,Parintins,33860,8260,0.0270,255,7228.30,223.15,0.0309,2021-02-21


In [71]:
casos_atual.columns = ['municipios', 'Notificados', 'Confirmados', 'Participacao','Obitos','Incidencia','Mortalidade','Letalidade','data_coleta']

In [72]:
casos_atual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   municipios    64 non-null     object        
 1   Notificados   64 non-null     int64         
 2   Confirmados   64 non-null     int64         
 3   Participacao  64 non-null     float64       
 4   Obitos        64 non-null     int64         
 5   Incidencia    64 non-null     float64       
 6   Mortalidade   64 non-null     float64       
 7   Letalidade    64 non-null     float64       
 8   data_coleta   64 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(3), object(1)
memory usage: 4.6+ KB


tratamento

In [73]:
casos_atual['Participacao'] = [x*100   for x in casos_atual['Participacao']]

In [74]:
casos_atual['Letalidade'] = [x*100   for x in casos_atual['Letalidade']]

In [75]:
casos_atual.columns = ['municipios', 'Notificados', 'Confirmados (3)', '% de participação\npor Município','Nº de óbitos','(casos/100.000)','(óbitos/100.000)','Letalidade','data_coleta']

unpivot

In [76]:
df = casos_atual.melt (id_vars = ['data_coleta','municipios'], var_name = 'tipo_ocorrencia', value_name = 'qtd_casos')

In [77]:
df

,data_coleta,municipios,tipo_ocorrencia,qtd_casos
0,2021-02-21,AMAZONAS,Notificados,746528.00
1,2021-02-21,MANAUS,Notificados,384525.00
2,2021-02-21,INTERIOR,Notificados,362003.00
3,2021-02-21,Coari,Notificados,22800.00
4,2021-02-21,Parintins,Notificados,33860.00
...,...,...,...,...
443,2021-02-21,Japurá,Letalidade,0.93
444,2021-02-21,Careiro da Várzea,Letalidade,1.35
445,2021-02-21,Itamarati,Letalidade,1.18
446,2021-02-21,Codajás,Letalidade,2.52


In [82]:
df['tipo_ocorrencia'].value_counts()

Nº de óbitos                        64
Confirmados (3)                     64
(óbitos/100.000)                    64
Notificados                         64
(casos/100.000)                     64
% de participação\npor Município    64
Letalidade                          64
Name: tipo_ocorrencia, dtype: int64

###**anterior**

In [83]:
casos_anterior.head()

,Municípios,Notificados,Confirmados (3),% de participação\npor Município,Nº de óbitos,Incidência (casos/100.000),Mortalidade (óbitos/100.000),Letalidade,data_coleta
0,AMAZONAS,745154,305250,1.0000,10409,7365.01,251.15,0.0341,2021-02-20
1,MANAUS,383987,141374,0.4631,7370,6476.84,337.65,0.0521,2021-02-20
2,INTERIOR,361167,163876,0.5369,3039,8353.20,154.91,0.0185,2021-02-20
3,Coari,22750,8948,0.0293,167,10515.06,196.25,0.0187,2021-02-20
4,Parintins,33812,8243,0.0270,254,7213.43,222.27,0.0308,2021-02-20


tratamento

In [84]:
casos_anterior = casos_anterior.drop(columns=['data_coleta'])

In [85]:
casos_anterior.columns = ['municipios', 'Notificados', 'Confirmados', 'Participacao','Obitos','Incidencia','Mortalidade','Letalidade']

In [86]:
casos_anterior['Participacao'] = [x*100   for x in casos_anterior['Participacao']]

In [87]:
casos_anterior['Letalidade'] = [x*100   for x in casos_anterior['Letalidade']]

In [88]:
casos_anterior.columns = ['municipios', 'Notificados', 'Confirmados (3)', '% de participação\npor Município','Nº de óbitos','(casos/100.000)','(óbitos/100.000)','Letalidade']

unpivot

In [89]:
df2 = casos_anterior.melt (id_vars = 'municipios', var_name = 'tipo_ocorrencia', value_name = 'qtd_casos2')

In [90]:
df2

,municipios,tipo_ocorrencia,qtd_casos2
0,AMAZONAS,Notificados,745154.00
1,MANAUS,Notificados,383987.00
2,INTERIOR,Notificados,361167.00
3,Coari,Notificados,22750.00
4,Parintins,Notificados,33812.00
...,...,...,...
443,Japurá,Letalidade,0.93
444,Careiro da Várzea,Letalidade,1.35
445,Itamarati,Letalidade,1.18
446,Codajás,Letalidade,2.52


In [91]:
df2['tipo_ocorrencia'].value_counts()

Nº de óbitos                        64
Confirmados (3)                     64
(óbitos/100.000)                    64
Notificados                         64
(casos/100.000)                     64
% de participação\npor Município    64
Letalidade                          64
Name: tipo_ocorrencia, dtype: int64

###**gerando tabela casos acumulados**

In [109]:
casos_municipios = df

In [110]:
for index, row in casos_municipios.iterrows():

    if row['tipo_ocorrencia'] in ['(óbitos/100.000)','Letalidade']:
      casos_municipios.at[index,'modalidade'] = 'Mortalidade'
    elif row['tipo_ocorrencia'] == '(casos/100.000)':
      casos_municipios.at[index,'modalidade'] = 'Incidência'
    else:
      casos_municipios.at[index,'modalidade'] = 'Casos'

In [111]:
casos_municipios['data_coleta'] = [str(x).replace(' ','T')  for x in casos_municipios['data_coleta']]

In [113]:
casos_municipios = casos_municipios.reindex(columns=['municipios','qtd_casos','tipo_ocorrencia','modalidade','data_coleta'])

In [114]:
casos_municipios.head()

,municipios,qtd_casos,tipo_ocorrencia,modalidade,data_coleta
0,AMAZONAS,746528.0,Notificados,Casos,2021-02-21T00:00:00
1,MANAUS,384525.0,Notificados,Casos,2021-02-21T00:00:00
2,INTERIOR,362003.0,Notificados,Casos,2021-02-21T00:00:00
3,Coari,22800.0,Notificados,Casos,2021-02-21T00:00:00
4,Parintins,33860.0,Notificados,Casos,2021-02-21T00:00:00


In [115]:
casos_municipios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448 entries, 0 to 447
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   municipios       448 non-null    object 
 1   qtd_casos        448 non-null    float64
 2   tipo_ocorrencia  448 non-null    object 
 3   modalidade       448 non-null    object 
 4   data_coleta      448 non-null    object 
dtypes: float64(1), object(4)
memory usage: 17.6+ KB


In [118]:
casos_municipios.to_json(f'covid_fvs_casos_por_municipios.json',orient="records")
casos_municipios.to_excel(f'covid_fvs_casos_por_municipios.xlsx')

###**gerando tabela historico de casos - tratamento e calculo**

In [92]:
df3 = df.merge(df2, how='inner', on=['municipios','tipo_ocorrencia'])

In [93]:
df3

,data_coleta,municipios,tipo_ocorrencia,qtd_casos,qtd_casos2
0,2021-02-21,AMAZONAS,Notificados,746528.00,745154.00
1,2021-02-21,MANAUS,Notificados,384525.00,383987.00
2,2021-02-21,INTERIOR,Notificados,362003.00,361167.00
3,2021-02-21,Coari,Notificados,22800.00,22750.00
4,2021-02-21,Parintins,Notificados,33860.00,33812.00
...,...,...,...,...,...
443,2021-02-21,Japurá,Letalidade,0.93,0.93
444,2021-02-21,Careiro da Várzea,Letalidade,1.35,1.35
445,2021-02-21,Itamarati,Letalidade,1.18,1.18
446,2021-02-21,Codajás,Letalidade,2.52,2.52


In [94]:
df3['tipo_ocorrencia'].value_counts()

Nº de óbitos                        64
Confirmados (3)                     64
(óbitos/100.000)                    64
Notificados                         64
(casos/100.000)                     64
% de participação\npor Município    64
Letalidade                          64
Name: tipo_ocorrencia, dtype: int64

In [95]:
for index, row in df3.iterrows():
    df3.at[index,'dados_diarios'] = row['qtd_casos'] - row['qtd_casos2']
    if row['tipo_ocorrencia'] in ['(óbitos/100.000)','Letalidade']:
      df3.at[index,'modalidade'] = 'Mortalidade'
    elif row['tipo_ocorrencia'] == '(casos/100.000)':
      df3.at[index,'modalidade'] = 'Incidência'
    else:
      df3.at[index,'modalidade'] = 'Casos'


In [96]:
df3

,data_coleta,municipios,tipo_ocorrencia,qtd_casos,qtd_casos2,dados_diarios,modalidade
0,2021-02-21,AMAZONAS,Notificados,746528.00,745154.00,1374.0,Casos
1,2021-02-21,MANAUS,Notificados,384525.00,383987.00,538.0,Casos
2,2021-02-21,INTERIOR,Notificados,362003.00,361167.00,836.0,Casos
3,2021-02-21,Coari,Notificados,22800.00,22750.00,50.0,Casos
4,2021-02-21,Parintins,Notificados,33860.00,33812.00,48.0,Casos
...,...,...,...,...,...,...,...
443,2021-02-21,Japurá,Letalidade,0.93,0.93,0.0,Mortalidade
444,2021-02-21,Careiro da Várzea,Letalidade,1.35,1.35,0.0,Mortalidade
445,2021-02-21,Itamarati,Letalidade,1.18,1.18,0.0,Mortalidade
446,2021-02-21,Codajás,Letalidade,2.52,2.52,0.0,Mortalidade


In [97]:
df3 = df3.drop(columns=['qtd_casos2'])

In [98]:
df3.columns = ['data_coleta', 'municipios', 'tipo_ocorrencia', 'dados_acumulados','dados_diarios','modalidade']

In [100]:
df3['data_coleta'] = [str(x).replace(' ','T')  for x in df3['data_coleta']]

In [102]:
historico_casos = df3.reindex(columns=['municipios','dados_acumulados','dados_diarios','tipo_ocorrencia','modalidade','data_coleta'])

In [103]:
historico_casos.head()

,municipios,dados_acumulados,dados_diarios,tipo_ocorrencia,modalidade,data_coleta
0,AMAZONAS,746528.0,1374.0,Notificados,Casos,2021-02-21T00:00:00
1,MANAUS,384525.0,538.0,Notificados,Casos,2021-02-21T00:00:00
2,INTERIOR,362003.0,836.0,Notificados,Casos,2021-02-21T00:00:00
3,Coari,22800.0,50.0,Notificados,Casos,2021-02-21T00:00:00
4,Parintins,33860.0,48.0,Notificados,Casos,2021-02-21T00:00:00


In [105]:
historico_casos['tipo_ocorrencia'].value_counts()

Nº de óbitos                        64
Confirmados (3)                     64
(óbitos/100.000)                    64
Notificados                         64
(casos/100.000)                     64
% de participação\npor Município    64
Letalidade                          64
Name: tipo_ocorrencia, dtype: int64

In [104]:
historico_casos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 448 entries, 0 to 447
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   municipios        448 non-null    object 
 1   dados_acumulados  448 non-null    float64
 2   dados_diarios     448 non-null    float64
 3   tipo_ocorrencia   448 non-null    object 
 4   modalidade        448 non-null    object 
 5   data_coleta       448 non-null    object 
dtypes: float64(2), object(4)
memory usage: 44.5+ KB


In [107]:
historico_casos.to_json(f'covid_fvs_historico_casos.json',orient="records")
historico_casos.to_excel(f'covid_fvs_historico_casos.xlsx')

##**Vacinados**

###**Por grupo**

In [ ]:
vacinados_por_grupo.head()

,TIPO,TOTAL,DATA COLETA
0,TRABALHADORES DA SAÚDE,55183,2021-02-21
1,POVOS INDIGENAS,234,2021-02-21
2,PESSOAS COM 60 ANOS OU MAIS INSTITUCIONALIZADAS,157,2021-02-21
3,PESSOAS COM DEFICIÊNCIA,94,2021-02-21
4,PESSOAS COM 80 ANOS OU MAIS,17845,2021-02-21


In [ ]:
vacinados_por_grupo.columns = ['grupo', 'total_vacinados', 'data_coleta']

In [ ]:
'''
vacinados_por_grupo['datatime_coleta'] = time_stamp()

vacinados_por_grupo['datatime_coleta'] = pd.to_datetime( vacinados_por_grupo['datatime_coleta'])
# Coluna 'Data'
vacinados_por_grupo['data'] = vacinados_por_grupo['datatime_coleta'].dt.strftime('%Y-%m-%d')
# Coluna 'Hora'
vacinados_por_grupo['hora'] = vacinados_por_grupo['datatime_coleta'].dt.strftime('%H:%M:%S')
'''

"\nvacinados_por_grupo['datatime_coleta'] = time_stamp()\n\nvacinados_por_grupo['datatime_coleta'] = pd.to_datetime( vacinados_por_grupo['datatime_coleta'])\n# Coluna 'Data'\nvacinados_por_grupo['data'] = vacinados_por_grupo['datatime_coleta'].dt.strftime('%Y-%m-%d')\n# Coluna 'Hora'\nvacinados_por_grupo['hora'] = vacinados_por_grupo['datatime_coleta'].dt.strftime('%H:%M:%S')\n"

In [ ]:
vacinados_por_grupo['grupo'] = vacinados_por_grupo['grupo'].fillna(value = 'Não Informado').astype('str')
vacinados_por_grupo['total_vacinados'] = vacinados_por_grupo['total_vacinados'].fillna(value =0).astype('int')
vacinados_por_grupo['data_coleta'] = [str(x).replace(' ','T')  for x in vacinados_por_grupo['data_coleta']]


In [ ]:
vacinados_por_grupo.dtypes

grupo              object
total_vacinados     int64
data_coleta        object
dtype: object

In [ ]:
vacinados_por_grupo.head()

,grupo,total_vacinados,data_coleta
0,TRABALHADORES DA SAÚDE,55183,2021-02-21T00:00:00
1,POVOS INDIGENAS,234,2021-02-21T00:00:00
2,PESSOAS COM 60 ANOS OU MAIS INSTITUCIONALIZADAS,157,2021-02-21T00:00:00
3,PESSOAS COM DEFICIÊNCIA,94,2021-02-21T00:00:00
4,PESSOAS COM 80 ANOS OU MAIS,17845,2021-02-21T00:00:00


In [ ]:
vacinados_por_grupo.to_json(f'covid_prefeitura_vacinados_por_grupo.json',orient="records")

###**Por dia**

In [ ]:
vacinados_por_dia


,DATA,TOTAL
0,2021-01-19,539
1,2021-01-20,4686
2,2021-01-22,423
3,2021-01-23,754
4,2021-01-24,41
5,2021-01-25,864
6,2021-01-26,276
7,2021-01-27,566
8,2021-01-28,571
9,2021-01-29,3439


In [ ]:
vacinados_por_dia.columns = ['dia', 'total_vacinados']

In [ ]:
vacinados_por_dia['dia'] = [str(x).replace(' ','T')  for x in vacinados_por_dia['dia']]
vacinados_por_dia['total_vacinados'] = vacinados_por_dia['total_vacinados'].fillna(value = 0).astype('int')
#vacinados_por_dia['data_coleta'] = [str(x).replace(' ','T')  for x in vacinados_por_dia['data_coleta']]
#vacinados_por_dia['datatime_coleta'] = [str(x).replace(' ','T') for x in vacinados_por_dia['datatime_coleta']]
#vacinados_por_dia['data'] = vacinados_por_dia['data'].fillna(value = '2000-01-01')



In [ ]:
vacinados_por_dia.dtypes

dia                object
total_vacinados     int64
dtype: object

In [ ]:
vacinados_por_dia.head()

,dia,total_vacinados
0,2021-01-19T00:00:00,539
1,2021-01-20T00:00:00,4686
2,2021-01-22T00:00:00,423
3,2021-01-23T00:00:00,754
4,2021-01-24T00:00:00,41


In [ ]:
vacinados_por_dia.to_json(f'covid_prefeitura_vacinados_por_dia.json',orient="records")

###**Por Local**

In [ ]:
vacinados_por_local.head()

,LOCAL,CATEGORIA,GRUPO,TOTAL,DATA COLETA
0,LESTE - EM DOMICÍLIO,Faixa Etária,Pessoas de 70 a 74 anos,42,2021-02-21
1,LESTE - EM DOMICÍLIO,Faixa Etária,Pessoas de 65 a 69 anos,1,2021-02-21
2,LESTE - EM DOMICÍLIO,Trabalhadores de Saúde,Outros,1,2021-02-21
3,LESTE - EM DOMICÍLIO,Faixa Etária,Pessoas de 75 a 79 anos,45,2021-02-21
4,LESTE - EM DOMICÍLIO,Faixa Etária,Pessoas de 80 anos ou mais,99,2021-02-21


In [ ]:
vacinados_por_local.columns = ['local', 'categoria', 'grupo', 'total','data_coleta']

In [ ]:
'''
vacinados_por_local['datatime_coleta'] = time_stamp()

vacinados_por_local['datatime_coleta'] = pd.to_datetime( vacinados_por_local['datatime_coleta'])
# Coluna 'Data'
vacinados_por_local['data'] = vacinados_por_local['datatime_coleta'].dt.strftime('%Y-%m-%d')
# Coluna 'Hora'
vacinados_por_local['hora'] = vacinados_por_local['datatime_coleta'].dt.strftime('%H:%M:%S')
'''

"\nvacinados_por_local['datatime_coleta'] = time_stamp()\n\nvacinados_por_local['datatime_coleta'] = pd.to_datetime( vacinados_por_local['datatime_coleta'])\n# Coluna 'Data'\nvacinados_por_local['data'] = vacinados_por_local['datatime_coleta'].dt.strftime('%Y-%m-%d')\n# Coluna 'Hora'\nvacinados_por_local['hora'] = vacinados_por_local['datatime_coleta'].dt.strftime('%H:%M:%S')\n"

In [ ]:
vacinados_por_local['local'] = vacinados_por_local['local'].fillna(value = 'Não Informado')
vacinados_por_local['categoria'] = vacinados_por_local['categoria'].fillna(value = 'Não Informado')
vacinados_por_local['grupo'] = vacinados_por_local['grupo'].fillna(value = 'Não Informado')
vacinados_por_local['total'] = vacinados_por_local['total'].fillna(value = 0)



In [ ]:
vacinados_por_local['local'] = vacinados_por_local['local'].astype('str')
vacinados_por_local['categoria'] = vacinados_por_local['categoria'].astype('str')
vacinados_por_local['grupo'] = vacinados_por_local['grupo'].astype('str')
vacinados_por_local['total'] = vacinados_por_local['total'].astype('int')
vacinados_por_local['data_coleta'] = [str(x).replace(' ','T')  for x in vacinados_por_local['data_coleta']]


In [ ]:
vacinados_por_local.dtypes

local          object
categoria      object
grupo          object
total           int64
data_coleta    object
dtype: object

In [ ]:
vacinados_por_local.head()

,local,categoria,grupo,total,data_coleta
0,LESTE - EM DOMICÍLIO,Faixa Etária,Pessoas de 70 a 74 anos,42,2021-02-21T00:00:00
1,LESTE - EM DOMICÍLIO,Faixa Etária,Pessoas de 65 a 69 anos,1,2021-02-21T00:00:00
2,LESTE - EM DOMICÍLIO,Trabalhadores de Saúde,Outros,1,2021-02-21T00:00:00
3,LESTE - EM DOMICÍLIO,Faixa Etária,Pessoas de 75 a 79 anos,45,2021-02-21T00:00:00
4,LESTE - EM DOMICÍLIO,Faixa Etária,Pessoas de 80 anos ou mais,99,2021-02-21T00:00:00


In [ ]:
vacinados_por_local.to_json(f'covid_prefeitura_vacinados_por_local.json',orient="records")

###**Totais**

In [ ]:
vacinados_totais.head()

,TIPO,TOTAL,DATA COLETA
0,TOTAL DE VACINADOS,121453,2021-02-21
1,LOCAIS VACINADOS,97,2021-02-21
2,TRABALHADORES DE SAÚDE,55183,2021-02-21
3,VACINADOS 1º DOSE,113953,2021-02-21
4,VACINADOS 2º DOSE,7500,2021-02-21


In [ ]:
vacinados_totais.columns = ['tipo', 'total','data_coleta']

In [ ]:
'''
vacinados_totais['datatime_coleta'] = time_stamp()

vacinados_totais['datatime_coleta'] = pd.to_datetime( casos_municipios['datatime_coleta'])
# Coluna 'Data'
vacinados_totais['data'] = casos_municipios['datatime_coleta'].dt.strftime('%Y-%m-%d')
# Coluna 'Hora'
vacinados_totais['hora'] = casos_municipios['datatime_coleta'].dt.strftime('%H:%M:%S')
'''

"\nvacinados_totais['datatime_coleta'] = time_stamp()\n\nvacinados_totais['datatime_coleta'] = pd.to_datetime( casos_municipios['datatime_coleta'])\n# Coluna 'Data'\nvacinados_totais['data'] = casos_municipios['datatime_coleta'].dt.strftime('%Y-%m-%d')\n# Coluna 'Hora'\nvacinados_totais['hora'] = casos_municipios['datatime_coleta'].dt.strftime('%H:%M:%S')\n"

In [ ]:
vacinados_totais['tipo'] = vacinados_totais['tipo'].fillna(value = 'Não Informado').astype('str')
vacinados_totais['total'] = vacinados_totais['total'].fillna(value = 0).astype('int')
vacinados_totais['data_coleta'] = [str(x).replace(' ','T')  for x in vacinados_totais['data_coleta']]



In [ ]:
vacinados_totais.dtypes

tipo           object
total           int64
data_coleta    object
dtype: object

In [ ]:
vacinados_totais.head()

,tipo,total,data_coleta
0,TOTAL DE VACINADOS,121453,2021-02-21T00:00:00
1,LOCAIS VACINADOS,97,2021-02-21T00:00:00
2,TRABALHADORES DE SAÚDE,55183,2021-02-21T00:00:00
3,VACINADOS 1º DOSE,113953,2021-02-21T00:00:00
4,VACINADOS 2º DOSE,7500,2021-02-21T00:00:00


In [ ]:
vacinados_totais.to_json(f'covid_prefeitura_vacinados_totais.json',orient="records")

##**Sepultamentos**

In [ ]:
sepultamentos.head()

,REDE PUBLICA,PARTICULAR,TOTAL,DATA
0,46,12,58,2021-01-01
1,66,10,76,2021-01-02
2,50,22,72,2021-01-03
3,52,14,66,2021-01-04
4,72,19,91,2021-01-05


In [ ]:
sepultamentos.columns = ['rede_publica', 'particular', 'total','data_coleta']

In [ ]:
sepultamentos['rede_publica'] = sepultamentos['rede_publica'].astype('int')
sepultamentos['particular'] = sepultamentos['particular'].astype('int')
sepultamentos['total'] = sepultamentos['total'].astype('int')
sepultamentos['data_coleta'] = [str(x).replace(' ','T')  for x in sepultamentos['data_coleta']]

In [ ]:
sepultamentos.dtypes

rede_publica     int64
particular       int64
total            int64
data_coleta     object
dtype: object

In [ ]:
sepultamentos.head()

,rede_publica,particular,total,data_coleta
0,46,12,58,2021-01-01T00:00:00
1,66,10,76,2021-01-02T00:00:00
2,50,22,72,2021-01-03T00:00:00
3,52,14,66,2021-01-04T00:00:00
4,72,19,91,2021-01-05T00:00:00


In [ ]:
sepultamentos.columns = ['REDE PUBLICA', 'PARTICULAR', 'TOTAL','data_coleta']

In [ ]:
sepultamentos.to_json(f'covid_numero_sepultamentos_por_rede.json',orient="records")

##**Leitos**

###Leitos

In [ ]:
leitos.head()

,REDE,CATEGORIA,TOTAL,PACIENTES ATE 20 DIAS,PACIENTES ACIMA DE 20 DIAS,TIPO DE LEITO,CATEGORIA DE PACIENTE,DATA COLETA
0,REDE PUBLICA,GERAL,290,201,89,UTI,CONFIRMADOS,2021-02-21
1,REDE PUBLICA,GERAL,538,410,128,CLINICO,CONFIRMADOS,2021-02-21
2,REDE PUBLICA,GERAL,14,0,0,SALA VERMELHA,CONFIRMADOS,2021-02-21
3,REDE PUBLICA,GESTANTE,13,13,0,UTI,CONFIRMADOS,2021-02-21
4,REDE PUBLICA,GESTANTE,20,20,0,CLINICO,CONFIRMADOS,2021-02-21


In [ ]:
leitos.columns = ['rede', 'tipo_hospital', 'total','pacientes_ate_20_dias', 'pacientes_acima_20_dias','tipo_leito', 'situacao_caso', 'data_coleta']

In [ ]:
leitos['tipo_hospital'] = leitos['tipo_hospital'].str.replace('GESTANTE','MATERNIDADE')

In [ ]:
leitos['rede'] = leitos['rede'].astype('str')
leitos['tipo_hospital'] = leitos['tipo_hospital'].astype('str')
leitos['total'] = leitos['total'].astype('int')
leitos['pacientes_ate_20_dias'] = leitos['pacientes_ate_20_dias'].astype('int')
leitos['pacientes_acima_20_dias'] = leitos['pacientes_acima_20_dias'].astype('int')
leitos['tipo_leito'] = leitos['tipo_leito'].astype('str')
leitos['situacao_caso'] = leitos['situacao_caso'].astype('str')
leitos['data_coleta'] = [str(x).replace(' ','T')  for x in leitos['data_coleta']]

In [ ]:
leitos.dtypes

rede                       object
tipo_hospital              object
total                       int64
pacientes_ate_20_dias       int64
pacientes_acima_20_dias     int64
tipo_leito                 object
situacao_caso              object
data_coleta                object
dtype: object

In [ ]:
leitos.head()

,rede,tipo_hospital,total,pacientes_ate_20_dias,pacientes_acima_20_dias,tipo_leito,situacao_caso,data_coleta
0,REDE PUBLICA,GERAL,290,201,89,UTI,CONFIRMADOS,2021-02-21T00:00:00
1,REDE PUBLICA,GERAL,538,410,128,CLINICO,CONFIRMADOS,2021-02-21T00:00:00
2,REDE PUBLICA,GERAL,14,0,0,SALA VERMELHA,CONFIRMADOS,2021-02-21T00:00:00
3,REDE PUBLICA,MATERNIDADE,13,13,0,UTI,CONFIRMADOS,2021-02-21T00:00:00
4,REDE PUBLICA,MATERNIDADE,20,20,0,CLINICO,CONFIRMADOS,2021-02-21T00:00:00


In [ ]:
leitos_privado = leitos[(leitos['rede']=='REDE PRIVADA')]

In [ ]:
leitos_privado

,rede,tipo_hospital,total,pacientes_ate_20_dias,pacientes_acima_20_dias,tipo_leito,situacao_caso,data_coleta
21,REDE PRIVADA,PRIVADO,192,174,18,UTI,CONFIRMADOS,2021-02-21T00:00:00
22,REDE PRIVADA,PRIVADO,188,176,12,CLINICO,CONFIRMADOS,2021-02-21T00:00:00
23,REDE PRIVADA,PRIVADO,0,0,0,SALA VERMELHA,CONFIRMADOS,2021-02-21T00:00:00
45,REDE PRIVADA,PRIVADO,26,26,0,UTI,SUSPEITOS,2021-02-21T00:00:00
46,REDE PRIVADA,PRIVADO,39,39,0,CLINICO,SUSPEITOS,2021-02-21T00:00:00
47,REDE PRIVADA,PRIVADO,4,0,0,SALA VERMELHA,SUSPEITOS,2021-02-21T00:00:00


In [ ]:
leitos.to_json(f'covid_fvs_leitos.json',orient="records")

###Taxa de Ocupação

In [ ]:
taxa_ocupacao.head()

,REDE,CATEGORIA,TAXA GERAL,TAXA COVID,TIPO DE LEITO,DATA COLETA
0,REDE PUBLICA,GERAL,80.00,97.56,UTI,2021-02-21
1,REDE PUBLICA,GERAL,85.79,85.37,CLINICO,2021-02-21
2,REDE PUBLICA,GERAL,97.30,75.56,SALA VERMELHA,2021-02-21
3,REDE PUBLICA,GESTANTE,91.80,100.00,UTI,2021-02-21
4,REDE PUBLICA,GESTANTE,82.40,24.10,CLINICO,2021-02-21


In [ ]:
taxa_ocupacao.columns = ['rede', 'tipo_hospital', 'taxa_geral','taxa_covid', 'tipo_leito','data_coleta']

In [ ]:
taxa_ocupacao['tipo_hospital'] = taxa_ocupacao['tipo_hospital'].str.replace('GESTANTE','MATERNIDADE')

In [ ]:
taxa_ocupacao['rede'] = taxa_ocupacao['rede'].astype('str')
taxa_ocupacao['tipo_hospital'] = taxa_ocupacao['tipo_hospital'].astype('str')
taxa_ocupacao['taxa_geral'] = taxa_ocupacao['taxa_geral'].astype('float')
taxa_ocupacao['taxa_covid'] = taxa_ocupacao['taxa_covid'].astype('float')
taxa_ocupacao['tipo_leito'] = taxa_ocupacao['tipo_leito'].astype('str')
taxa_ocupacao['data_coleta'] = [str(x).replace(' ','T')  for x in taxa_ocupacao['data_coleta']]

In [ ]:
taxa_ocupacao.dtypes

rede              object
tipo_hospital     object
taxa_geral       float64
taxa_covid       float64
tipo_leito        object
data_coleta       object
dtype: object

In [ ]:
taxa_ocupacao.head()

,rede,tipo_hospital,taxa_geral,taxa_covid,tipo_leito,data_coleta
0,REDE PUBLICA,GERAL,80.00,97.56,UTI,2021-02-21T00:00:00
1,REDE PUBLICA,GERAL,85.79,85.37,CLINICO,2021-02-21T00:00:00
2,REDE PUBLICA,GERAL,97.30,75.56,SALA VERMELHA,2021-02-21T00:00:00
3,REDE PUBLICA,MATERNIDADE,91.80,100.00,UTI,2021-02-21T00:00:00
4,REDE PUBLICA,MATERNIDADE,82.40,24.10,CLINICO,2021-02-21T00:00:00


In [ ]:
taxa_ocupacao.to_json(f'covid_fvs_leitos_taxa_ocupacao.json',orient="records")

###Capacidade Instalada

In [ ]:
capacidade_instalada.head()

,REDE,CATEGORIA,OCUPADOS,LIVRE,TIPO DE LEITO,CATEGORIA DO LEITO,DATA COLETA
0,REDE PUBLICA,GERAL,28,7,UTI,GERAL,2021-02-21
1,REDE PUBLICA,GERAL,338,56,CLINICO,GERAL,2021-02-21
2,REDE PUBLICA,GERAL,72,2,SALA VERMELHA,GERAL,2021-02-21
3,REDE PUBLICA,GESTANTE,56,5,UTI,GERAL,2021-02-21
4,REDE PUBLICA,GESTANTE,426,91,CLINICO,GERAL,2021-02-21


In [ ]:
capacidade_instalada.columns = ['rede', 'tipo_hospital', 'leitos_ocupados','leitos_livres', 'tipo_leito','categoria_leito','data_coleta']

In [ ]:
capacidade_instalada['tipo_hospital'] = capacidade_instalada['tipo_hospital'].str.replace('GESTANTE','MATERNIDADE')

In [ ]:
capacidade_instalada['rede'] = capacidade_instalada['rede'].astype('str')
capacidade_instalada['tipo_hospital'] = capacidade_instalada['tipo_hospital'].astype('str')
capacidade_instalada['leitos_ocupados'] = capacidade_instalada['leitos_ocupados'].astype('int')
capacidade_instalada['leitos_livres'] = capacidade_instalada['leitos_livres'].astype('int')
capacidade_instalada['tipo_leito'] = capacidade_instalada['tipo_leito'].astype('str')
capacidade_instalada['categoria_leito'] = capacidade_instalada['categoria_leito'].astype('str')
capacidade_instalada['data_coleta'] = [str(x).replace(' ','T')  for x in capacidade_instalada['data_coleta']]

In [ ]:
capacidade_instalada.dtypes

rede               object
tipo_hospital      object
leitos_ocupados     int64
leitos_livres       int64
tipo_leito         object
categoria_leito    object
data_coleta        object
dtype: object

In [ ]:
capacidade_privado = capacidade_instalada[(capacidade_instalada['rede']=='REDE PRIVADA')]

In [ ]:
capacidade_privado

,rede,tipo_hospital,leitos_ocupados,leitos_livres,tipo_leito,categoria_leito,data_coleta
21,REDE PRIVADA,PRIVADO,83,25,UTI,GERAL,2021-02-21T00:00:00
22,REDE PRIVADA,PRIVADO,249,122,CLINICO,GERAL,2021-02-21T00:00:00
23,REDE PRIVADA,PRIVADO,2,4,SALA VERMELHA,GERAL,2021-02-21T00:00:00
45,REDE PRIVADA,PRIVADO,218,30,UTI,COVID-19,2021-02-21T00:00:00
46,REDE PRIVADA,PRIVADO,227,97,CLINICO,COVID-19,2021-02-21T00:00:00
47,REDE PRIVADA,PRIVADO,4,2,SALA VERMELHA,COVID-19,2021-02-21T00:00:00


In [ ]:
capacidade_instalada.head()

,rede,tipo_hospital,leitos_ocupados,leitos_livres,tipo_leito,categoria_leito,data_coleta
0,REDE PUBLICA,GERAL,28,7,UTI,GERAL,2021-02-21T00:00:00
1,REDE PUBLICA,GERAL,338,56,CLINICO,GERAL,2021-02-21T00:00:00
2,REDE PUBLICA,GERAL,72,2,SALA VERMELHA,GERAL,2021-02-21T00:00:00
3,REDE PUBLICA,MATERNIDADE,56,5,UTI,GERAL,2021-02-21T00:00:00
4,REDE PUBLICA,MATERNIDADE,426,91,CLINICO,GERAL,2021-02-21T00:00:00


In [ ]:
capacidade_instalada.to_json(f'covid_fvs_leitos_capacidade_instalada.json',orient="records")

##**Predição Casos**

In [ ]:
predicao_casos.head()

,Unnamed: 0,date,media_movel_casos_prevista
0,0,01/11,321.53537
1,1,02/11,293.61417
2,2,03/11,279.36307
3,3,04/11,260.54446
4,4,05/11,258.25810


In [ ]:
predicao_casos = predicao_casos.drop(columns=['Unnamed: 0'])

In [ ]:
predicao_casos.columns = ['data', 'media_movel_casos']

**Tratamento data**

In [ ]:

predicao_casos["data"] =['{}-2020'.format(x.replace('/','-')) if x[3:6] == '12' or x[3:6] == '11'
              else '{}-2021'.format(x.replace('/','-'))
              for x in predicao_casos['data']]

In [ ]:
predicao_casos["data"] = [datetime.strptime(x,'%d-%m-%Y').date() for x in predicao_casos["data"]]

In [ ]:
predicao_casos.head()

,data,media_movel_casos
0,2020-11-01,321.53537
1,2020-11-02,293.61417
2,2020-11-03,279.36307
3,2020-11-04,260.54446
4,2020-11-05,258.25810


In [ ]:
#predicao_casos['media_movel_casos'] = predicao_casos['media_movel_casos'].astype('float')
predicao_casos['data'] = [str(x) + ' 00:00:00'  for x in predicao_casos['data'] ]

In [ ]:
predicao_casos["data"] = [datetime.strptime(x,'%Y-%m-%d %H:%M:%S') for x in predicao_casos["data"]]

In [ ]:
predicao_casos["data"] = [str(x).replace(' ','T')  for x in predicao_casos["data"]]

In [ ]:
#predicao_casos["data"] = [datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%f") for x in predicao_casos["data"]]

In [ ]:
predicao_casos.dtypes

data                  object
media_movel_casos    float64
dtype: object

In [ ]:
predicao_casos.head()

,data,media_movel_casos
0,2020-11-01T00:00:00,321.53537
1,2020-11-02T00:00:00,293.61417
2,2020-11-03T00:00:00,279.36307
3,2020-11-04T00:00:00,260.54446
4,2020-11-05T00:00:00,258.25810


In [ ]:
predicao_casos.to_json(f'covid_predicao_casos.json',orient="records")

##**Predição Óbitos**

In [ ]:
predicao_obitos.head()

,Unnamed: 0,date,media_movel_deaths_prevista
0,0,01/11,9.903850
1,1,02/11,9.981517
2,2,03/11,10.041626
3,3,04/11,9.393461
4,4,05/11,8.992658


In [ ]:
predicao_obitos = predicao_obitos.drop(columns=['Unnamed: 0'])

In [ ]:
predicao_obitos.columns = ['data', 'media_movel_obitos']

**Tratamento data**

In [ ]:

predicao_obitos["data"] =['{}-2020'.format(x.replace('/','-')) if x[3:6] == '12' or x[3:6] == '11'
              else '{}-2021'.format(x.replace('/','-'))
              for x in predicao_obitos['data']]

In [ ]:
predicao_obitos["data"] = [datetime.strptime(x,'%d-%m-%Y').date() for x in predicao_obitos["data"]]

In [ ]:
predicao_obitos.head()

,data,media_movel_obitos
0,2020-11-01,9.903850
1,2020-11-02,9.981517
2,2020-11-03,10.041626
3,2020-11-04,9.393461
4,2020-11-05,8.992658


In [ ]:
#predicao_obitos['media_movel_obitos'] = predicao_obitos['media_movel_obitos'].astype('float')
predicao_obitos['data'] = [str(x) + ' 00:00:00'  for x in predicao_obitos['data'] ]

In [ ]:
predicao_obitos["data"] = [datetime.strptime(x,'%Y-%m-%d %H:%M:%S') for x in predicao_obitos["data"]]

In [ ]:
predicao_obitos["data"] = [str(x).replace(' ','T')  for x in predicao_obitos["data"]]

In [ ]:
#predicao_casos["data"] = [datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%f") for x in predicao_casos["data"]]

In [ ]:
predicao_obitos.dtypes

data                   object
media_movel_obitos    float64
dtype: object

In [ ]:
predicao_obitos.head()

,data,media_movel_obitos
0,2020-11-01T00:00:00,9.903850
1,2020-11-02T00:00:00,9.981517
2,2020-11-03T00:00:00,10.041626
3,2020-11-04T00:00:00,9.393461
4,2020-11-05T00:00:00,8.992658


In [ ]:
predicao_obitos.to_json(f'covid_predicao_obitos.json',orient="records")

#**Envio**

##**Casos**

###**Historica**

In [108]:
f = open ('covid_fvs_historico_casos.json', "r", encoding = "utf-8") 
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'covid_fvs_historico_casos',
    'id': f"Item{cont}{i['data_coleta']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1


helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(448, [])

###**Diária**

In [ ]:
f = open ('covid_fvs_casos_por_municipios.json', "r", encoding = "utf-8") 
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'covid_fvs_casos_por_municipios',
    'id': f"Item{cont}{i['data_coleta']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1


helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(448, [])

##**Vacinados**

###**Diária**

In [ ]:
f = open ('covid_prefeitura_vacinados_por_dia.json', "r", encoding = "utf-8") 
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'covid_prefeitura_vacinados_por_dia',
    'id': f"Item{cont}{i['dia']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1


helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(32, [])

In [ ]:
f = open ('covid_prefeitura_vacinados_por_grupo.json', "r", encoding = "utf-8") 
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'covid_prefeitura_vacinados_por_grupo',
    'id': f"Item{cont}{i['data_coleta']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1


helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(12, [])

In [ ]:
f = open ('covid_prefeitura_vacinados_por_local.json', "r", encoding = "utf-8") 
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'covid_prefeitura_vacinados_por_local',
    'id': f"Item{cont}{i['data_coleta']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1


helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(981, [])

In [ ]:
f = open ('covid_prefeitura_vacinados_totais.json', "r", encoding = "utf-8") 
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'covid_prefeitura_vacinados_totais',
    'id': f"Item{cont}{i['data_coleta']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1


helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(5, [])

##**Sepultamentos**

In [ ]:
f = open ('covid_numero_sepultamentos_por_rede.json', "r", encoding = "utf-8") 
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'covid_numero_sepultamentos_por_rede',
    'id': f"Item{cont}{i['data_coleta']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1


helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(52, [])

##**Leitos**

In [ ]:
f = open ('covid_fvs_leitos.json', "r", encoding = "utf-8") 
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'covid_fvs_leitos',
    'id': f"Item{cont}{i['data_coleta']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1


helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(48, [])

In [ ]:
f = open ('covid_fvs_leitos_taxa_ocupacao.json', "r", encoding = "utf-8") 
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'covid_fvs_leitos_taxa_ocupacao',
    'id': f"Item{cont}{i['data_coleta']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1


helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(24, [])

In [ ]:
f = open ('covid_fvs_leitos_capacidade_instalada.json', "r", encoding = "utf-8") 
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'covid_fvs_leitos_capacidade_instalada',
    'id': f"Item{cont}{i['data_coleta']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1


helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(48, [])

##**Tabelas de Predição**

###**Predição casos**


In [ ]:
f = open ('covid_predicao_casos.json', "r", encoding = "utf-8") 
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'covid_predicao_casos',
    'id': f"Item{cont}{i['data']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1


helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(121, [])

###**Predição obitos**


In [ ]:
f = open ('covid_predicao_obitos.json', "r", encoding = "utf-8") 
data = json.loads(f.read())
itens = []
cont = 0

for i in data:
  dict = {
    '_index': 'covid_predicao_obitos',
    'id': f"Item{cont}{i['data']}",
    '_source': i
  }

  itens.append(dict)
  cont+=1


helpers.bulk(es,itens)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


(121, [])

*Não Mexer nessa base*